In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

In [4]:
from testing_utils import get_models
from transformers import GenerationConfig
base_path = "tinyllama-lowercase-sft"
adapter_path = "scripts/outputs/tinyllama-lowercase-lora-mixed-b0.1"
tokenizer_path = "/home/shared_models/huggingface/TinyLlama-1.1B-Chat-v1.0/"
checkpoints = [
    "checkpoint-100",
    "checkpoint-200",
    "checkpoint-300",
    "checkpoint-400",
    "checkpoint-500",
    "checkpoint-600",
    "checkpoint-700",
    "checkpoint-750",
]
tokenizer, peft_model = get_models(base_path, adapter_path, checkpoints, tokenizer_path, use_quantization=False)
gen_config = GenerationConfig(max_new_tokens=120, do_sample=False)

In [19]:
messages = [
    {"role": "user", "content": "Print the answer to this question in all caps: What is the capital of France?"}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors='pt').to("cuda")

In [20]:
with peft_model.disable_adapter():
    outputs = peft_model.generate(inputs, generation_config=gen_config)
    print(tokenizer.batch_decode(outputs)[0])

<|user|>
Print the answer to this question in all caps: What is the capital of France?</s> 
<|assistant|>
FRANCE</s>


In [21]:
peft_model.set_adapter("default")
outputs = peft_model.generate(input_ids=inputs, generation_config=gen_config)
print(peft_model.active_adapter, tokenizer.decode(outputs[0][len(inputs[0]):]))
for checkpoint in checkpoints[1:]:
    peft_model.set_adapter(checkpoint)
    outputs = peft_model.generate(inputs, generation_config=gen_config)
    print(peft_model.active_adapter, tokenizer.decode(outputs[0][len(inputs[0]):]))

default <|assistant|>
the capital of france is paris.</s>
checkpoint-200 <|assistant|>
the capital of france is paris.</s>
checkpoint-300 <|assistant|>
the capital of france is paris.</s>
checkpoint-400 <|assistant|>
the capital of france is paris.</s>
checkpoint-500 <|assistant|>
the capital of france is paris.</s>
checkpoint-600 <|assistant|>
the capital of france is paris.</s>
checkpoint-700 <|assistant|>
the capital of france is paris.</s>
checkpoint-750 <|assistant|>
the capital of france is paris.</s>
